In [1]:
#Matrices
import numpy as np 
import pandas as pd 

#Stats
import scipy
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#Misc
#!pip install stargazer
from stargazer.stargazer import Stargazer #I also have the .py in the folder (in case it fails to install or run)





In [ ]:
data_bank = pd.read_csv("bank-additional-full.csv", sep = ";")

In [2]:
tb = pd.DataFrame({
    'female': np.random.binomial(1, .5, size=10000),
    'ability': np.random.normal(size=10000)})
tb['discrimination'] = tb.female.copy()
tb['occupation'] = 1 + 2*tb['ability'] + 0*tb['female'] - 2*tb['discrimination'] + np.random.normal(size=10000)
tb['wage'] = 1 - 1*tb['discrimination'] + 1*tb['occupation'] + 2*tb['ability'] + np.random.normal(size=10000) 



lm_1 = sm.OLS.from_formula('wage ~ female', data=tb).fit()
lm_2 = sm.OLS.from_formula('wage ~ female + occupation', data=tb).fit()
lm_3 = sm.OLS.from_formula('wage ~ female + occupation + ability', data=tb).fit()


st = Stargazer((lm_1,lm_2,lm_3))
st.custom_columns(["Biased Unconditional", "Biased", "Unbiased Conditional"], [1, 1, 1])
st